In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [ ]:
train_df.head()

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,TRAIN_0000,2009,CT005,이커머스,Series A,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,0.3
1,TRAIN_0001,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,0.8
2,TRAIN_0002,2018,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.00,3500-4500,0.5
3,TRAIN_0003,2016,CT006,NaN,Seed,3245.0,Yes,Yes,NaN,665.0,10547.0,2.97,NaN,0.7
4,TRAIN_0004,2020,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.00,1500-2500,0.1


In [ ]:
test_df.head()

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원)
0,TEST_0000,2002,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,2.00,1500-2500
1,TEST_0001,2020,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,4.20,NaN
2,TEST_0002,2014,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,1.00,6000이상
3,TEST_0003,2003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,5.00,1500-2500
4,TEST_0004,2006,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,4.36,NaN


In [ ]:
train_df.isnull().sum()

,0
ID,0
설립연도,0
국가,0
분야,857
투자단계,0
직원 수,174
인수여부,0
상장여부,0
고객수(백만명),1320
총 투자금(억원),0


In [ ]:
test_df.isnull().sum()

,0
ID,0
설립연도,0
국가,0
분야,354
투자단계,0
직원 수,76
인수여부,0
상장여부,0
고객수(백만명),547
총 투자금(억원),0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   국가              4376 non-null   object 
 3   분야              3519 non-null   object 
 4   투자단계            4376 non-null   object 
 5   직원 수            4202 non-null   float64
 6   인수여부            4376 non-null   object 
 7   상장여부            4376 non-null   object 
 8   고객수(백만명)        3056 non-null   float64
 9   총 투자금(억원)       4376 non-null   float64
 10  연매출(억원)         4376 non-null   float64
 11  SNS 팔로워 수(백만명)  4376 non-null   float64
 12  기업가치(백억원)       3156 non-null   object 
 13  성공확률            4376 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 478.8+ KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              1755 non-null   object 
 1   설립연도            1755 non-null   int64  
 2   국가              1755 non-null   object 
 3   분야              1401 non-null   object 
 4   투자단계            1755 non-null   object 
 5   직원 수            1679 non-null   float64
 6   인수여부            1755 non-null   object 
 7   상장여부            1755 non-null   object 
 8   고객수(백만명)        1208 non-null   float64
 9   총 투자금(억원)       1755 non-null   float64
 10  연매출(억원)         1755 non-null   float64
 11  SNS 팔로워 수(백만명)  1755 non-null   float64
 12  기업가치(백억원)       1268 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 178.4+ KB


In [ ]:
# "ID" 컬럼 처리
test_id = test_df[["ID"]]
train_df.drop(columns=["ID"], inplace=True)
test_df.drop(columns=["ID"], inplace=True)

In [ ]:
# X, y 분리
X = train_df.drop(columns=["성공확률"])
y = train_df["성공확률"]

In [ ]:
def parse_range(val):
    if isinstance(val, str):
        val = val.strip()
        if '-' in val:  # 예: '2500-3500'
            start, end = val.split('-')
            return (float(start) + float(end)) / 2
        elif '이상' in val:  # 예: '6000이상'
            return float(val.replace('이상', '').strip())
    return val

X["기업가치(백억원)"] = X["기업가치(백억원)"].apply(parse_range)
test_df["기업가치(백억원)"] = test_df["기업가치(백억원)"].apply(parse_range)

In [ ]:
X.nunique()

,0
설립연도,23
국가,10
분야,10
투자단계,5
직원 수,2884
인수여부,2
상장여부,2
고객수(백만명),99
총 투자금(억원),3184
연매출(억원),3716


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   설립연도            4376 non-null   int64  
 1   국가              4376 non-null   object 
 2   분야              3519 non-null   object 
 3   투자단계            4376 non-null   object 
 4   직원 수            4202 non-null   float64
 5   인수여부            4376 non-null   object 
 6   상장여부            4376 non-null   object 
 7   고객수(백만명)        3056 non-null   float64
 8   총 투자금(억원)       4376 non-null   float64
 9   연매출(억원)         4376 non-null   float64
 10  SNS 팔로워 수(백만명)  4376 non-null   float64
 11  기업가치(백억원)       3156 non-null   float64
dtypes: float64(6), int64(1), object(5)
memory usage: 410.4+ KB


In [ ]:
X["분야"].fillna("Unknown", inplace=True)
test_df["분야"].fillna("Unknown", inplace=True)

<ipython-input-14-979435bb8ce2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X["분야"].fillna("Unknown", inplace=True)
<ipython-input-14-979435bb8ce2>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met

In [ ]:
from sklearn.preprocessing import LabelEncoder

binary_cols = ["인수여부", "상장여부"]
le = LabelEncoder()

for col in binary_cols:
    X[col] = le.fit_transform(X[col])
    test_df[col] = le.transform(test_df[col])

In [ ]:
X_encoded = pd.get_dummies(X, columns=["국가", "분야", "투자단계"])
test_df_encoded = pd.get_dummies(test_df, columns=["국가", "분야", "투자단계"])

# train/test 컬럼 수 맞추기
X_encoded, test_df_encoded = X_encoded.align(test_df_encoded, join='left', axis=1, fill_value=0)

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=10)
X_imputed = knn_imputer.fit_transform(X_encoded)
test_imputed = knn_imputer.transform(test_df_encoded)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
test_scaled = scaler.transform(test_imputed)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# 모델 학습
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# WMAE 계산 (가중치 임의 예시)
weights = np.ones_like(y_test)  # 가중치가 전부 1이면 일반 MAE와 동일
wmae = np.sum(weights * np.abs(y_test - y_pred)) / np.sum(weights)
print("WMAE:", wmae)

WMAE: 0.2018687214611872


In [ ]:
test_pred = model.predict(test_scaled)
submission = test_id.copy()
submission["성공확률"] = test_pred

# CSV 파일로 저장
submission.to_csv("submission.csv", index=False)